IMPORTANT:this TF file Needs MP_Data folder from data collection step

In [72]:
#To remove leftovers from previous training
#!rm -rf MP_Data_Child
#!rm -rf MP_Data_Child.zip
#!rm -rf action.h5
#!rm -rf Logs

# Preprocess

In [73]:
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [74]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
#import mediapipe as mp

In [75]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
#Upload the zipped data
#!unzip file_location
!unzip MP_Data_Child.zip

In [77]:
# Path for exported data, numpy arrays
Folder='MP_Data_Child'

DATA_PATH = os.path.join(Folder) 

#Sorting to negate shuffling of dirs
dirs=os.listdir(Folder)
dirs.sort()

# Actions that we try to detect
actions = np.array(dirs)

# Thirty videos worth of data
no_sequences = len(os.listdir(os.path.join(Folder,actions[0])))

# Videos are going to be 24 frames in length
sequence_length = len(os.listdir(os.path.join(Folder,actions[0],str(0))))


In [78]:
label_map = {label:num for num, label in enumerate(actions)}

In [79]:
label_map

{'B': 0, 'C': 1, 'L': 2}

In [80]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [81]:
#was able to condense info from 1662 datapoints to 261

In [82]:
#np.array(labels)

In [83]:
X = np.array(sequences)
X.shape

(300, 24, 225)

In [84]:
y = to_categorical(labels).astype(int)
#y

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [86]:
X_train.shape, X_test.shape

((270, 24, 225), (30, 24, 225))

# Train

In [87]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

TensorBoard

In [88]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

LSTM Training Model

In [89]:
X.shape

(300, 24, 225)

In [90]:
model = Sequential()
#3 sets of LSTM layers
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(X.shape[1],X.shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
#2 Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#Softmax function
model.add(Dense(actions.shape[0], activation='softmax'))

In [91]:
#(X.shape[1],X.shape[2])

In [92]:
#actions.shape[0]

In [93]:
#sidework
#res = [.7, .3]
#actions[np.argmax(res)]

In [94]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [95]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
9/9 [==============================] - 4s 70ms/step - loss: 1.1123 - categorical_accuracy: 0.2704
Epoch 2/200
9/9 [==============================] - 1s 82ms/step - loss: 1.0990 - categorical_accuracy: 0.3407
Epoch 3/200
9/9 [==============================] - 1s 97ms/step - loss: 1.0949 - categorical_accuracy: 0.3889
Epoch 4/200
9/9 [==============================] - 1s 66ms/step - loss: 1.0930 - categorical_accuracy: 0.4037
Epoch 5/200
9/9 [==============================] - 1s 64ms/step - loss: 1.0835 - categorical_accuracy: 0.4593
Epoch 6/200
9/9 [==============================] - 1s 63ms/step - loss: 1.1310 - categorical_accuracy: 0.4667
Epoch 7/200
9/9 [==============================] - 1s 64ms/step - loss: 1.0866 - categorical_accuracy: 0.4296
Epoch 8/200
9/9 [==============================] - 1s 62ms/step - loss: 1.0685 - categorical_accuracy: 0.5185
Epoch 9/200
9/9 [==============================] - 1s 63ms/step - loss: 1.0553 - categorical_accuracy: 0.3926
Epoch 10/2

Epoch 200/200
15/15 [==============================] - 1s 68ms/step - loss: 0.0180 - categorical_accuracy: 0.9911

In [96]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 24, 64)            74240     
                                                                 
 lstm_4 (LSTM)               (None, 24, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 228,803
Trainable params: 228,803
Non-tr

In [97]:
#tensorboard log dir
#!zip -r /content/file.zip /content/Logs
#from google.colab import files
#files.download("/content/file.zip")
#go to the correct command directory for Logs and type in the following in the command line..
#cd ..filedir
#tensorboard --logdir=.
#..then copy pase the URL to a browser

# Testing

In [98]:
result=model.predict(X_test)

In [99]:
actions[np.argmax(result[0])]

'L'

In [100]:
#Self evaluation
#i=0
#(actions[np.argmax(result[i])]==
#actions[np.argmax(y_test[i])])

In [101]:
model.save('action.h5')

In [102]:
#to save to local device
#from google.colab import files
#files.download("/content/action.h5")

In [103]:
#To delete model
#del model

In [104]:
model.load_weights('action.h5')

# Evaluation

In [105]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [106]:
yhat = model.predict(X_test)

In [107]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [108]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[21,  1],
        [ 1,  7]],

       [[15,  1],
        [ 1, 13]],

       [[21,  1],
        [ 1,  7]]])

In [110]:
accuracy_score(ytrue, yhat)

0.9